In [2]:
import pandas as pd
import numpy as np
import scipy as sp

In [3]:
datos = pd.read_excel('datos_socioeconomicos_ingresantes.xlsx',skiprows=[0,1] )

In [4]:
columnas = ['nombre_carrera', 'sexo', 'obra_social', 'madre_trabajo', 'madre_vive', 'padre_trabajo', 
            'padre_vive', 'trabajo', 'horas_sem_trabajo', 'padre_ult_est_curs', 
            'madre_ult_est_curs', 'nota_matematica', 'matcursprisem','cantmataprob_al_21-10-2017',
            'dep_practica', 'idioma_ingles', 'idioma_franc', 'idioma_portu', 'idioma_itali', 'idioma_aleman']

In [5]:
datos = datos[columnas]

    nombre_carrera= se tranformó a código numérico - sin Nans
    sexo= se binarizó - sin Nans
    matcursprisem= se transformó a números - sin Nans
    cantmataprob...= se transformó a números - sin Nans

    obra_social= vectorización(binario) - 9 Nans
    madre_trabajo= vectorización(binario) - 492 Nans
    padre_trabajo= vectorización(binario) - 590 Nans
    trabajo= vectorización(binario) - 5 Nans

    horas_sem_trabajo= vectorización(binario) - 2199 Nans

    madre_vive= binarización - 4 Nans
    padre_vive= binarización - 5 Nans
    padre_ult_est_curs= vectorización - 7 Nans
    madre_ult_est_curs= vectorización - 5 Nans
    dep_practica= binarización 1 Nan
    idioma_ingles = 0, 1, 2 (categórica ordinal) - sin Nans

    nota_matematica = 1088 Nans 

Tratamiento de los nulos:

 - Se eliminan los nulos de 'madre_vive','padre_vive','padre_ult_est_curs','madre_ult_est_curs', 'dep_practica','trabajo', 'obra_social'
 - Para las variables madre_trabajo y padre_trabajo, se transforman los nulos en la categoría Desconoce.
 - Para la variable nota_matemática, se transforman los nulos en no aprobados.
 - La variable cantidad de horas de trabajo tiene 56 nulos que se corresponden con la categoría:"trabajó al menos una hora" de la variable trabajo (eso es el 10% de la población total que trabaja). Por otro lado, todos los que tienen un valor en la variable "horas de trabajo" pertenecen a la categoría "trabajó al menos una hora", así que eso es coherente.  

#### Sacar los nulls de columnas: madre_vive, padre_vive, padre_ult_est_curs, madre_ult_est_curs y dep_practica, trabajo, obra social

In [6]:
datos_null = datos.dropna(subset=['madre_vive','padre_vive','padre_ult_est_curs','madre_ult_est_curs',
                                  'dep_practica','trabajo', 'obra_social'])

#### Transformar nota matematica a 1(aprobado) 0 (no aprobado)

In [7]:
def trans_matem(val):
    if val=="A":
        return(1)
    return(0)

In [8]:
datos_null.nota_matematica=datos_null.nota_matematica.apply(trans_matem)

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/pandas/core/generic.py:5209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


#### Transformar categorías de estudios padre madre en números.

In [9]:
#definir números para las categorías. Es el agrupamiento que hacen en el anuario UNC
def from_estudio_s(value):
    if 'de postgrado' in value.lower():
        #return 'postgrado'
        return 4
    if 'universitarios completos' in value.lower():
        #return 'univ_completos'
        return 4
    if 'universitarios incompletos' in value.lower():
        #return 'univ_incompleto'
        return 4
    if 'superiores completos' in value.lower():
        #return 'sup_completo'
        return 4
    if 'superiores incompletos' in value.lower():
        #return 'sup_incompleto'
        return 4
    if 'secundarios completos' in value.lower():
        #return 'sec_completo'
        return 3
    if 'secundarios incompletos' in value.lower() or 'primarios completos' in value.lower():
        #return 'primario_completo-sec_inc'
        return 2
    if 'primarios incompletos' in value.lower() or 'no hizo estudios' in value.lower():
        #return 'primario_inc-sin_estudios'
        return 1
    #return 'Desconoce'
    return 0

In [10]:
#definir números para las categorías. Es el agrupamiento que hacen en el anuario UNC
def from_estudio(value):
    if 'de postgrado' in value.lower():
        #return 'postgrado'
        return 8
    if 'universitarios completos' in value.lower():
        #return 'univ_completos'
        return 7
    if 'universitarios incompletos' in value.lower():
        #return 'univ_incompleto'
        return 6
    if 'superiores completos' in value.lower():
        #return 'sup_completo'
        return 5
    if 'superiores incompletos' in value.lower():
        #return 'sup_incompleto'
        return 4
    if 'secundarios completos' in value.lower():
        #return 'sec_completo'
        return 3
    if 'secundarios incompletos' in value.lower() or 'primarios completos' in value.lower():
        #return 'primario_completo-sec_inc'
        return 2
    if 'primarios incompletos' in value.lower() or 'no hizo estudios' in value.lower():
        #return 'primario_inc-sin_estudios'
        return 1
    #return 'Desconoce'
    return 0

In [11]:
def add_col(data, col_ori, col_nva, transf):
    data.loc[data.index, col_nva] = data[col_ori].apply(transf)

In [12]:
add_col(datos_null, "padre_ult_est_curs", "estudios_padre", from_estudio)
add_col(datos_null, "madre_ult_est_curs", "estudios_madre", from_estudio)

add_col(datos_null, "padre_ult_est_curs", "estudios_padre_s", from_estudio_s)
add_col(datos_null, "madre_ult_est_curs", "estudios_madre_s", from_estudio_s)

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [13]:
# binarizamos nivel de estudios con una cantidad de categorías reducida: primaria incompleta; 
#primaria completa/sec incompleta; sec completa; superiores comp/incomp univ compl/inc o postgrado

nv_estudios_madre_s = pd.get_dummies(datos_null.estudios_madre_s, prefix='niv_est_madre', prefix_sep='_')
datos_null = pd.merge(datos_null, nv_estudios_madre_s, left_index=True, right_index=True)
nv_estudios_padre_s = pd.get_dummies(datos_null.estudios_padre_s, prefix='niv_est_padre', prefix_sep='_')
datos_null = pd.merge(datos_null, nv_estudios_padre_s, left_index=True, right_index=True)

#### Transformar categorías de idiomas en enteros

In [14]:
def from_idioma(value):
    if 'Muy bueno' in value:
        return 3
    if 'Bueno' in value:
        return 2
    if 'Básico' in value:
        return 1
    return 0

In [15]:
datos_null.loc[datos_null.index, 'idioma_ingles'] = datos_null.loc[
                                                    datos_null.index, 'idioma_ingles'].apply(from_idioma)

#### Construir variables binarias con las categorías de 'trabajo'

In [16]:
col_trabajo = pd.get_dummies(datos_null.trabajo, prefix='alm', prefix_sep='_')

In [17]:
datos_null = pd.merge(datos_null, col_trabajo, left_index=True, right_index=True)

In [18]:
datos_null.madre_trabajo = datos_null.madre_trabajo.fillna('Desconoce')
datos_null.padre_trabajo = datos_null.padre_trabajo.fillna('Desconoce')

In [19]:
col_trabajo_madre = pd.get_dummies(datos_null.madre_trabajo, prefix='madre', prefix_sep='_')

In [20]:
col_trabajo_padre = pd.get_dummies(datos_null.padre_trabajo, prefix='padre', prefix_sep='_')

In [21]:
datos_null = pd.merge(datos_null, col_trabajo_madre, left_index=True, right_index=True)

In [22]:
datos_null = pd.merge(datos_null, col_trabajo_padre, left_index=True, right_index=True)

#### Construir variable binaria con obra social

In [23]:
col_obrasocial = pd.get_dummies(datos_null.obra_social)

In [24]:
datos_null = pd.merge(datos_null,col_obrasocial,left_index=True, right_index=True)

#### Construir variable binaria con horas semanales de trabajo

In [25]:
col_horastrabajo = pd.get_dummies(datos_null.horas_sem_trabajo, prefix='alm', prefix_sep='_')

In [26]:
datos_null = pd.merge(datos_null,col_horastrabajo,left_index=True, right_index=True) 

#### Binarizar variables SI NO

In [27]:
def trans_SN(val):
    if val=="S":
        return(1)
    if val=="N":
        return(0)
    return(0)

In [28]:
datos_null.dep_practica = datos_null.dep_practica.apply(trans_SN)

In [29]:
datos_null.madre_vive = datos_null.madre_vive.apply(trans_SN)

In [30]:
datos_null.padre_vive = datos_null.padre_vive.apply(trans_SN)

#### Transfomar string -> int

In [31]:
datos_null.loc[datos_null.index, 'sexo'] = pd.to_numeric(datos_null.loc[datos_null.index, 'sexo'])

In [32]:
datos_null.loc[datos_null.index, 'matcursprisem'] = pd.to_numeric(datos_null.loc[datos_null.index, 'matcursprisem'])

In [33]:
datos_null.loc[datos_null.index, 'cantmataprob_al_21-10-2017'] = pd.to_numeric(datos_null.loc[
               datos_null.index, 'cantmataprob_al_21-10-2017'])

#### Transformar carreras a variables numéricas

In [34]:
datos_null.nombre_carrera = pd.Categorical(datos_null.nombre_carrera)

In [35]:
datos_null.loc[ datos_null.index, 'cod_carrera'] = datos_null.nombre_carrera.cat.codes

In [36]:
_df = pd.DataFrame({'a': [1,2,3,4,5,6,7,8], 'b':[8,7,6,5,4,3,2,1]})

In [37]:
_df.apply(lambda x: x[0] + x[1], axis=1)

0    9
1    9
2    9
3    9
4    9
5    9
6    9
7    9
dtype: int64

#### Juntar otros idiomas (excluyendo ingles) y hacer categorías numéricas

In [38]:
datos_otros_idms = datos_null.loc[datos_null.index,['idioma_itali', 'idioma_franc', 'idioma_aleman', 'idioma_portu']]

In [39]:
def juntar_sel(l):
    lstring = ' '.join(l)
    if 'Muy bueno' in lstring:
        return 3
    if 'Bueno' in lstring:
        return 2
    if 'Básico' in lstring:
        return 1
    return 0

In [40]:
datos_null.loc[datos_null.index, 'otros_idms'] = datos_otros_idms.apply(juntar_sel, axis=1)

In [41]:
datos_null.otros_idms.value_counts()

0    2013
1     522
2     172
Name: otros_idms, dtype: int64

In [42]:
# binarizamos nivel de inglés generando tres variables, cada una por categoría de nivel de inglés
nv_ingles = pd.get_dummies(datos_null.idioma_ingles, prefix='niv_ingles', prefix_sep='_')
datos_null = pd.merge(datos_null, nv_ingles, left_index=True, right_index=True)

### Columnas que seleccionamos después de la exploración

In [43]:
cols_sel = [ 
      'nota_matematica',
      'madre_vive',
      'padre_vive',
      'alm_No trabajó y buscó trabajo en algún momento de los últimos 30 días',
      'alm_No trabajó y no buscó trabajo', 
      'alm_Trabajó al menos una hora',
      'madre_Desconoce',
      'madre_No trabajó y buscó trabajo en algún momento de los últimos 30 días',
      'madre_No trabajó y no buscó trabajo',
      'madre_Trabajó al menos una hora', 
      'padre_Desconoce',
      'padre_No trabajó y buscó trabajo en algún momento de los últimos 30 días',
      'padre_No trabajó y no buscó trabajo',
      'padre_Trabajó al menos una hora', 
      'Carece de Obra Social',
      'Como afiliado voluntario', 
      'Otorgada por la universidad',
      'Por ser familiar a cargo', 
      'Por su trabajo',
      'niv_est_madre_0',
      'niv_est_madre_1', 
      'niv_est_madre_2',
      'niv_est_madre_3', 
      'niv_est_madre_4', 
      'niv_est_padre_0',
      'niv_est_padre_1', 
      'niv_est_padre_2', 
      'niv_est_padre_3',
      'niv_est_padre_4', 
      'niv_ingles_0', 
      'niv_ingles_1', 
      'niv_ingles_2'
        ]

# sacamos las variables sexo, otros idiomas, dep_practica porque no vemos influencia en la variable dependiente
# y la carrera porque habría que reagrupar o segmentar para usarla en la regresión.

X_no = [ 'sexo', 'otros_idms', 'dep_practica', 'cod_carrera']

# usamos como y la nota de matemática pero vimos que se ven cosas muy similares con 
# 'matcursprisem' y 'cantmataprob_al_21-10-2017'.

#y = ['nota_matematica']

#cambiar el nombre de algunas columnas:
data_sel = datos_null[cols_sel]

dic_columnas = {'alm_No trabajó y buscó trabajo en algún momento de los últimos 30 días': 'alm_NoTr_B',
'alm_No trabajó y no buscó trabajo': 'alm_NoTr_NoB',
'alm_Trabajó al menos una hora': 'alm_Tr',
'madre_No trabajó y buscó trabajo en algún momento de los últimos 30 días': 'madre_NoTr_B',
'madre_No trabajó y no buscó trabajo': 'madre_NoTr_NoB',
'madre_Trabajó al menos una hora': 'madre_Tr',
'padre_No trabajó y buscó trabajo en algún momento de los últimos 30 días': 'padre_NoTr_B',
'padre_No trabajó y no buscó trabajo': 'padre_NoTr_NoB',
'padre_Trabajó al menos una hora': 'padre_Tr',
'Carece de Obra Social' : 'OS_sin',
'Como afiliado voluntario': 'OS_voluntario',
'Otorgada por la universidad': 'OS_univ',
'Por ser familiar a cargo': 'OS_familiar',
'Por su trabajo':  'OS_trabajo'}

data_sel.rename(columns=dic_columnas, inplace=True)
#data_sel

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/pandas/core/frame.py:4218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [44]:
print(datos_null.columns)
print(data_sel.columns)

Index(['nombre_carrera', 'sexo', 'obra_social', 'madre_trabajo', 'madre_vive',
       'padre_trabajo', 'padre_vive', 'trabajo', 'horas_sem_trabajo',
       'padre_ult_est_curs', 'madre_ult_est_curs', 'nota_matematica',
       'matcursprisem', 'cantmataprob_al_21-10-2017', 'dep_practica',
       'idioma_ingles', 'idioma_franc', 'idioma_portu', 'idioma_itali',
       'idioma_aleman', 'estudios_padre', 'estudios_madre', 'estudios_padre_s',
       'estudios_madre_s', 'niv_est_madre_0', 'niv_est_madre_1',
       'niv_est_madre_2', 'niv_est_madre_3', 'niv_est_madre_4',
       'niv_est_padre_0', 'niv_est_padre_1', 'niv_est_padre_2',
       'niv_est_padre_3', 'niv_est_padre_4',
       'alm_No trabajó y buscó trabajo en algún momento de los últimos 30 días',
       'alm_No trabajó y no buscó trabajo', 'alm_Trabajó al menos una hora',
       'madre_Desconoce',
       'madre_No trabajó y buscó trabajo en algún momento de los últimos 30 días',
       'madre_No trabajó y no buscó trabajo',
       '

# TP3 

### Análisis de los datos con distintos modelos: regresión logística, Perceptrón y SGD
## Regresión logística

#### Analisis de la correlación entre la condición de alumno trabajador y el desempeño, solo y en función de otras variables que serán indicadores de nivel y condiciones socieconómicas (madre_vive, padre_vive, estudios_padre, estudios_madre, trabajo_padre, trabajo_madre, nivel de inglés, obra social).

## Trabajo

- Pregunta: que los alumnos trabajen (y/o necesiten trabajar) afecta el desempeño,tomando como evaluador la aprobación de matemática del cursillo en el mismo año de ingreso.

Ajustamos un modelo donde la variable independiente es "trabajo" y la variable dependiente la nota de matemática. 
La variable tiene tres categorías: trabaja, no trabaja pero busca trabajo, no trabaja y no busca trabajo. Tomamos como referencia (o línea de base) la categoría que reúne alumnos que no trabajan ni buscan trabajo. Por eso solo nos quedamos con las dos variables binarias restantes. Observamos el comportamiento de esos dos grupos de alumnos en relación al primero.

In [45]:
from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from sklearn import metrics
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix

In [46]:
def logit_st(x, y):
    X_const = sm.add_constant(x)
    logit_model=sm.Logit(y,X_const) #[['niv_est_padre_1']])
    result=logit_model.fit()
    print(result.summary())
    logit_model.fit().params
    return()

In [47]:
def logit_sk(x, y):
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    logreg = LogisticRegression(fit_intercept = True, C = 1e9)
    m_fit = logreg.fit(x, y)
    print(m_fit)
    print('coeficientes= ',logreg.coef_)
    print('intercepto= ', logreg.intercept_)
    y_pred = logreg.predict(x)
    print('Accuracy en los datos: {:.2f}'.format(logreg.score(x, y)))
    #c_matrix = confusion_matrix(y, y_pred)
    #print(c_matrix) 
    return()

In [48]:
#ajustar con trabajo y ver qué pasa con trabajo y sumando de a una variable de por vez.
y_data = data_sel['nota_matematica']
x_data = data_sel[['alm_Tr' , 'alm_NoTr_B']]
logit_st(x_data, y_data)
logit_sk(x_data, y_data)

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


Optimization terminated successfully.
         Current function value: 0.628590
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:        nota_matematica   No. Observations:                 2707
Model:                          Logit   Df Residuals:                     2704
Method:                           MLE   Df Model:                            2
Date:                Mon, 26 Aug 2019   Pseudo R-squ.:                 0.06563
Time:                        21:03:32   Log-Likelihood:                -1701.6
converged:                       True   LL-Null:                       -1821.1
Covariance Type:            nonrobust   LLR p-value:                 1.244e-52
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9438      0.056     16.874      0.000       0.834       1.053
alm_Tr        -1.4212      0.

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
coeficientes=  [[-1.42123083 -1.00931324]]
intercepto=  [0.94383549]
Accuracy en los datos: 0.66


()

### resultado trabajo
Regresión logística en los datos de Trabajo, tomando como línea de base a los alumnos que no trabajan y no buscan trabajo, usando como variable dependiente la nota de Matemáticas del ciclo nivelador. Los coeficientes para la variación respecto a la condición "trabajar" y "no trabajar pero buscar trabajo", son ambos negativos y significativos (p<0.001). "La población que trabaja o busca trabajo tiene menos probabilidad (frecuencia) de aprobar". 
> Chequear : la variable trabajo es significativa para explicar el rendimiento (s/ nota de matemática), según el p-value del chi2 (LLR p-value: 1.244e-52)

## Madre vive / Padre vive

Pregunta: que la madre y el padre viva se correlaciona con el rendimiento de los alumnos (medido en función la variable nota_matematica)?

In [49]:
#ajustar con trabajo y ver qué pasa con trabajo y sumando de a una variable de por vez.
y_data = data_sel['nota_matematica']
x_data = data_sel[['madre_vive']]
logit_st(x_data, y_data)
logit_sk(x_data, y_data)

Optimization terminated successfully.
         Current function value: 0.669877
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:        nota_matematica   No. Observations:                 2707
Model:                          Logit   Df Residuals:                     2705
Method:                           MLE   Df Model:                            1
Date:                Mon, 26 Aug 2019   Pseudo R-squ.:                0.004257
Time:                        21:03:32   Log-Likelihood:                -1813.4
converged:                       True   LL-Null:                       -1821.1
Covariance Type:            nonrobust   LLR p-value:                 8.230e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5781      0.261     -2.219      0.026      -1.089      -0.067
madre_vive     1.0107      0.

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


()

In [50]:
y_data = data_sel['nota_matematica']
x_data = data_sel[['padre_vive']]
logit_st(x_data, y_data)
logit_sk(x_data, y_data)

Optimization terminated successfully.
         Current function value: 0.669165
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:        nota_matematica   No. Observations:                 2707
Model:                          Logit   Df Residuals:                     2705
Method:                           MLE   Df Model:                            1
Date:                Mon, 26 Aug 2019   Pseudo R-squ.:                0.005315
Time:                        21:03:32   Log-Likelihood:                -1811.4
converged:                       True   LL-Null:                       -1821.1
Covariance Type:            nonrobust   LLR p-value:                 1.083e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1487      0.133     -1.122      0.262      -0.408       0.111
padre_vive     0.6104      0.

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


()

In [51]:
y_data = data_sel['nota_matematica']
x_data = data_sel[['madre_vive', 'padre_vive']]
logit_st(x_data, y_data)
logit_sk(x_data, y_data)

Optimization terminated successfully.
         Current function value: 0.667332
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:        nota_matematica   No. Observations:                 2707
Model:                          Logit   Df Residuals:                     2704
Method:                           MLE   Df Model:                            2
Date:                Mon, 26 Aug 2019   Pseudo R-squ.:                0.008041
Time:                        21:03:33   Log-Likelihood:                -1806.5
converged:                       True   LL-Null:                       -1821.1
Covariance Type:            nonrobust   LLR p-value:                 4.371e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.8848      0.276     -3.205      0.001      -1.426      -0.344
madre_vive     0.8314      0.

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


()

### resultado madre o padre viven

En ambos casos los coeficientes son positivos y con p<0.001, e indican que la madre y el padre vivos, aumenta la chance de que los alumnos aprueben matemáticas. El coeficiente es mayor para la madre que para el padre. Los coeficientes son más pequeños que antes, como si hubiera dependencia entre ambas variables. Podría ser que la población de aquellos que perdieron al padre y la madre esté menos representada en la universidad (?). 

## Modelo que incluya las variables trabajo y madre vive / padre vive

Hacemos un modelo donde analizamos la nota de matemática, considerando si la alumna/o trabaja y si su madre y padre viven. De nuevo, la línea de base para trabajo son las alumnas que no trabajan ni buscan trabajo.

In [52]:
y_data = data_sel['nota_matematica']
x_data = data_sel[['alm_Tr' , 'alm_NoTr_B', 'madre_vive', 'padre_vive']]
logit_st(x_data, y_data)
logit_sk(x_data, y_data)

Optimization terminated successfully.
         Current function value: 0.626931
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:        nota_matematica   No. Observations:                 2707
Model:                          Logit   Df Residuals:                     2702
Method:                           MLE   Df Model:                            4
Date:                Mon, 26 Aug 2019   Pseudo R-squ.:                 0.06809
Time:                        21:03:33   Log-Likelihood:                -1697.1
converged:                       True   LL-Null:                       -1821.1
Covariance Type:            nonrobust   LLR p-value:                 1.743e-52
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1043      0.295      0.354      0.724      -0.473       0.682
alm_Tr        -1.3757      0.

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


()

### resultado trabajo, madre o padre viven

Los coeficientes asociados a que las alumnas trabajen o busquen trabajo, son similares a los originales (negativos ambos). Por otro lado, que la madre viva, aumenta la probabilidad de que la alumna/o apruebe matemática también en este modelo que incluye más variables. Eso significa que, al menos en parte, el efecto de la variable madre_viva es independiente de la variable trabajo. En cambio, en este modelo, el coeficiente asociado a que el padre esté vivo no es distinto de 0. En los gráficos de barras ya veíamos que esta variable estaba correlacionada con la condición de trabajador del alumno. Y según el modelo de regresión logística no aporta información independiente. En el caso de la madre, el coeficiente se hace más pequeño que antes, pero subsiste. 

## Nivel de estudios de la madre y del padre 

Ajustamos un modelo de regresión logística para "nota_matematica" (var. dependiente) y como variables independientes el nivel de estudios de la madre:
- 'niv_est_madre_0': desconoce
- 'niv_est_madre_1': primario incompleto 
- 'niv_est_madre_2': primario completo secundario incompleto
- 'niv_est_madre_3': secundario completo
- 'niv_est_madre_4': superior incompleto/completo, universitario incompleto/completo, posgrado.

Tomamos de línea de base el niv_est_madre_1

In [53]:
y_data = data_sel['nota_matematica']
x_data = data_sel[['niv_est_madre_0', 'niv_est_madre_2','niv_est_madre_3', 'niv_est_madre_4']]
logit_st(x_data, y_data)
logit_sk(x_data, y_data)

Optimization terminated successfully.
         Current function value: 0.641481
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:        nota_matematica   No. Observations:                 2707
Model:                          Logit   Df Residuals:                     2702
Method:                           MLE   Df Model:                            4
Date:                Mon, 26 Aug 2019   Pseudo R-squ.:                 0.04647
Time:                        21:03:33   Log-Likelihood:                -1736.5
converged:                       True   LL-Null:                       -1821.1
Covariance Type:            nonrobust   LLR p-value:                 1.522e-35
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.5845      0.214     -2.731      0.006      -1.004      -0.165
niv_est_madre_

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anac

()

In [54]:
y_data = data_sel['nota_matematica']
x_data = data_sel[['niv_est_padre_0', 'niv_est_padre_2','niv_est_padre_3', 'niv_est_padre_4']]
logit_st(x_data, y_data)
logit_sk(x_data, y_data)

Optimization terminated successfully.
         Current function value: 0.645182
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:        nota_matematica   No. Observations:                 2707
Model:                          Logit   Df Residuals:                     2702
Method:                           MLE   Df Model:                            4
Date:                Mon, 26 Aug 2019   Pseudo R-squ.:                 0.04097
Time:                        21:03:33   Log-Likelihood:                -1746.5
converged:                       True   LL-Null:                       -1821.1
Covariance Type:            nonrobust   LLR p-value:                 3.015e-31
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.6466      0.186     -3.475      0.001      -1.011      -0.282
niv_est_padre_

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anac

()

In [55]:
y_data = data_sel['nota_matematica']
x_data = data_sel[['niv_est_madre_0', 'niv_est_madre_2','niv_est_madre_3', 'niv_est_madre_4',
                   'niv_est_padre_0', 'niv_est_padre_2','niv_est_padre_3', 'niv_est_padre_4',]]
logit_st(x_data, y_data)
logit_sk(x_data, y_data)

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anac

Optimization terminated successfully.
         Current function value: 0.632485
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:        nota_matematica   No. Observations:                 2707
Model:                          Logit   Df Residuals:                     2698
Method:                           MLE   Df Model:                            8
Date:                Mon, 26 Aug 2019   Pseudo R-squ.:                 0.05984
Time:                        21:03:33   Log-Likelihood:                -1712.1
converged:                       True   LL-Null:                       -1821.1
Covariance Type:            nonrobust   LLR p-value:                 1.046e-42
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.8466      0.241     -3.518      0.000      -1.318      -0.375
niv_est_madre_

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


()

### resultado estudios, madre o padre

La categoria estudios_0 son los que registraron "desconoce", entonces es dificil analizarla. Tomamos como referencia (es decir, no incluimos la categoría estudios_1: primario incompleto). Respecto a ese grupo, el nivel de estudios de la madre y del padre aumenta la probabilidad de que el alumno apruebe matemática (los coeficientes son positivos y aumentan progresivamente con el nivel de estudios). En el caso de la madre no hay diferencia (significativa) entre las madres con primario incompleto y primario completo. En el caso del padre esos grupos sí son distintos.
Cuando incluímos todas las variables en el modelo, todos los coeficientes se hacen más pequeños (más próximos a 0). Eso significa que hay interdependencia entre las variables (el nivel de estudios de la madre y del padre están correlacionados). En este caso, niv_est_madre_1==niv_est_madre_2==niv_est_madre_3, y niv_est_padre_1==niv_est_padre_2

## Trabajo y nivel de estudios de la madre y del padre 


In [56]:
y_data = data_sel['nota_matematica']
x_data = data_sel[['alm_Tr' , 'alm_NoTr_B',
                   'niv_est_madre_0', 'niv_est_madre_2','niv_est_madre_3', 'niv_est_madre_4',
                   'niv_est_padre_0', 'niv_est_padre_2','niv_est_padre_3', 'niv_est_padre_4',]]
logit_st(x_data, y_data)
logit_sk(x_data, y_data)

Optimization terminated successfully.
         Current function value: 0.605127
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:        nota_matematica   No. Observations:                 2707
Model:                          Logit   Df Residuals:                     2696
Method:                           MLE   Df Model:                           10
Date:                Mon, 26 Aug 2019   Pseudo R-squ.:                  0.1005
Time:                        21:03:33   Log-Likelihood:                -1638.1
converged:                       True   LL-Null:                       -1821.1
Covariance Type:            nonrobust   LLR p-value:                 1.549e-72
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0367      0.256      0.144      0.886      -0.465       0.538
alm_Tr        

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anac

()

### resultado trabajo y estudios, madre o padre
En este caso, ya habíamos observado en los gráficos de barra que había correlación entre el nivel de estudio de la madre y padre y la condición de alumno trabajador. Vemos que al incluir todas las variables, se mantiene la tendencia con el nivel de estudios pero disminuye aún más el valor de los coeficientes. En el caso de las variables correspondiente a la madre ninguno es significativo (p<0.05). Para el padre sí (los alumnos con padres que terminan el secundario, y los que tienen estudios superiores, tienen más chance de aprobar matemáticas).

## Trabajo de la madre y del padre 
Tomamos como línea de base el grupo de madre o padres que no trabajan. La categoría desconoce no sabemos cómo está compuesta. 

In [57]:
y_data = data_sel['nota_matematica']
x_data = data_sel[['madre_Desconoce', 'madre_NoTr_B', 'madre_Tr']]
logit_st(x_data, y_data)
logit_sk(x_data, y_data)

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


Optimization terminated successfully.
         Current function value: 0.664005
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:        nota_matematica   No. Observations:                 2707
Model:                          Logit   Df Residuals:                     2703
Method:                           MLE   Df Model:                            3
Date:                Mon, 26 Aug 2019   Pseudo R-squ.:                 0.01299
Time:                        21:03:33   Log-Likelihood:                -1797.5
converged:                       True   LL-Null:                       -1821.1
Covariance Type:            nonrobust   LLR p-value:                 3.004e-10
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.0559      0.106     -0.528      0.597      -0.263       0.151
madre_Desconoc

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


()

In [58]:
y_data = data_sel['nota_matematica']
x_data = data_sel[['padre_Desconoce', 'padre_NoTr_B', 'padre_Tr']]
logit_st(x_data, y_data)
logit_sk(x_data, y_data)

Optimization terminated successfully.
         Current function value: 0.665925
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:        nota_matematica   No. Observations:                 2707
Model:                          Logit   Df Residuals:                     2703
Method:                           MLE   Df Model:                            3
Date:                Mon, 26 Aug 2019   Pseudo R-squ.:                 0.01013
Time:                        21:03:34   Log-Likelihood:                -1802.7
converged:                       True   LL-Null:                       -1821.1
Covariance Type:            nonrobust   LLR p-value:                 4.831e-08
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.1911      0.219     -0.872      0.383      -0.621       0.239
padre_Desconoc

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


()

In [59]:
y_data = data_sel['nota_matematica']
x_data = data_sel[['madre_Desconoce', 'madre_NoTr_B', 'madre_Tr',
                   'padre_Desconoce', 'padre_NoTr_B', 'padre_Tr']]
logit_st(x_data, y_data)
logit_sk(x_data, y_data)

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anac

Optimization terminated successfully.
         Current function value: 0.660412
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:        nota_matematica   No. Observations:                 2707
Model:                          Logit   Df Residuals:                     2700
Method:                           MLE   Df Model:                            6
Date:                Mon, 26 Aug 2019   Pseudo R-squ.:                 0.01833
Time:                        21:03:34   Log-Likelihood:                -1787.7
converged:                       True   LL-Null:                       -1821.1
Covariance Type:            nonrobust   LLR p-value:                 1.893e-12
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.4610      0.228     -2.018      0.044      -0.909      -0.013
madre_Desconoc

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


()

### resultado trabajo madre o padre
Que el padre y que la madre trabajen se correlaciona con mayor probabilidad de que la alumna/o aprueben matemática. No es diferente la condición de no trabajar a la de no trabajar y buscar trabajo. Cuando se incluye madre y padre, los coeficientes disminuyen (indicando que hay alguna correlación entre que el padre y que la madre tengan o no trabajo), pero siguen siendo positivos.

## Trabajo alumno y trabajo de la madre y del padre 

In [60]:
y_data = data_sel['nota_matematica']
x_data = data_sel[['alm_Tr' , 'alm_NoTr_B','padre_Desconoce', 'padre_NoTr_B', 'padre_Tr']]
logit_st(x_data, y_data)
logit_sk(x_data, y_data)

Optimization terminated successfully.
         Current function value: 0.625285
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:        nota_matematica   No. Observations:                 2707
Model:                          Logit   Df Residuals:                     2701
Method:                           MLE   Df Model:                            5
Date:                Mon, 26 Aug 2019   Pseudo R-squ.:                 0.07054
Time:                        21:03:34   Log-Likelihood:                -1692.6
converged:                       True   LL-Null:                       -1821.1
Covariance Type:            nonrobust   LLR p-value:                 1.790e-53
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.4200      0.233      1.801      0.072      -0.037       0.877
alm_Tr        

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anac

()

In [61]:
y_data = data_sel['nota_matematica']
x_data = data_sel[['alm_Tr' , 'alm_NoTr_B','madre_Desconoce', 'madre_NoTr_B', 'madre_Tr']]
logit_st(x_data, y_data)
logit_sk(x_data, y_data)

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anac

Optimization terminated successfully.
         Current function value: 0.623437
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:        nota_matematica   No. Observations:                 2707
Model:                          Logit   Df Residuals:                     2701
Method:                           MLE   Df Model:                            5
Date:                Mon, 26 Aug 2019   Pseudo R-squ.:                 0.07329
Time:                        21:03:34   Log-Likelihood:                -1687.6
converged:                       True   LL-Null:                       -1821.1
Covariance Type:            nonrobust   LLR p-value:                 1.275e-55
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.5123      0.117      4.367      0.000       0.282       0.742
alm_Tr        

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


()

In [62]:
y_data = data_sel['nota_matematica']
x_data = data_sel[['alm_Tr' , 'alm_NoTr_B',
                   'madre_Desconoce', 'madre_NoTr_B', 'madre_Tr',
                   'padre_Desconoce', 'padre_NoTr_B', 'padre_Tr']]
logit_st(x_data, y_data)
logit_sk(x_data, y_data)

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anac

Optimization terminated successfully.
         Current function value: 0.621718
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:        nota_matematica   No. Observations:                 2707
Model:                          Logit   Df Residuals:                     2698
Method:                           MLE   Df Model:                            8
Date:                Mon, 26 Aug 2019   Pseudo R-squ.:                 0.07584
Time:                        21:03:34   Log-Likelihood:                -1683.0
converged:                       True   LL-Null:                       -1821.1
Covariance Type:            nonrobust   LLR p-value:                 4.650e-55
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.1694      0.244      0.695      0.487      -0.308       0.647
alm_Tr        

()

### resultado trabajo alumno y madre o padre
Al incluir todas las variables (que en los gráficos de barra se ve que están correlacionadas), la dependencia entre "y" (nota_matemática) y que el padre y la madre tengan trabajo disminuye. En el caso del padre el coeficiente ya no es más significativo. En el caso de la madre sí. 

## Nivel de ingles
Tomamos como línea de bse el nivel de inglés 0 - no sabe inglés.

In [63]:
y_data = data_sel['nota_matematica']
x_data = data_sel[['niv_ingles_1' ,'niv_ingles_2' ]]
logit_st(x_data, y_data)
logit_sk(x_data, y_data)

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Optimization terminated successfully.
         Current function value: 0.654038
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:        nota_matematica   No. Observations:                 2707
Model:                          Logit   Df Residuals:                     2704
Method:                           MLE   Df Model:                            2
Date:                Mon, 26 Aug 2019   Pseudo R-squ.:                 0.02780
Time:                        21:03:34   Log-Likelihood:                -1770.5
converged:                       True   LL-Null:                       -1821.1
Covariance Type:            nonrobust   LLR p-value:                 1.028e-22
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.4595      0.184     -2.492      0.013      -0.821      -0.098
niv_ingles_1     0.6103

()

### resultado nivel de inglés
Hay una dependencia positiva entre el nivel de inglés y la nota en matemáticas (la probabilidad de aprobar es mayor en el grupo con mejor nivel de inglés). Nivel de inglés es probablemente un indicador del nivel socioeconómico.

## Trabajo del alumno y nivel de ingles

In [64]:
y_data = data_sel['nota_matematica']
x_data = data_sel[['alm_Tr' , 'alm_NoTr_B',
                   'niv_ingles_1' ,'niv_ingles_2' ]]
logit_st(x_data, y_data)
logit_sk(x_data, y_data)

Optimization terminated successfully.
         Current function value: 0.616539
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:        nota_matematica   No. Observations:                 2707
Model:                          Logit   Df Residuals:                     2702
Method:                           MLE   Df Model:                            4
Date:                Mon, 26 Aug 2019   Pseudo R-squ.:                 0.08354
Time:                        21:03:34   Log-Likelihood:                -1669.0
converged:                       True   LL-Null:                       -1821.1
Covariance Type:            nonrobust   LLR p-value:                 1.293e-64
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.1402      0.196      0.714      0.475      -0.245       0.525
alm_Tr          -1.3290

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anac

()

### resultado trabajo y nivel de inglés
Los valores absolutos de todos los coeficientes decaen, indicando que hay dependencia entre las condiciones pero siguen siendo positivos y significativos. 

## Obra Social
Tomamos como línea de la categoría OS a través de familiares

In [65]:
y_data = data_sel['nota_matematica']
x_data = data_sel[['OS_sin', 'OS_voluntario', 'OS_univ',
                   'OS_trabajo' ]]
logit_st(x_data, y_data)
logit_sk(x_data, y_data)


/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


Optimization terminated successfully.
         Current function value: 0.627471
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:        nota_matematica   No. Observations:                 2707
Model:                          Logit   Df Residuals:                     2702
Method:                           MLE   Df Model:                            4
Date:                Mon, 26 Aug 2019   Pseudo R-squ.:                 0.06729
Time:                        21:03:34   Log-Likelihood:                -1698.6
converged:                       True   LL-Null:                       -1821.1
Covariance Type:            nonrobust   LLR p-value:                 7.416e-52
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.8055      0.058     13.870      0.000       0.692       0.919
OS_sin           -1.

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


()

### resultado obra social
Aquellos que tienen obra social como afiliados voluntarios son similares a los que la tienen a través de un familiar. Para las otras categorías hay una disminución en la probabilidad de aprobar matemática, que es mayor (coef más negativo) para los que tienen OS a través del trabajo, luego para los que no tienen OS, y finalmente para los que la tienen a través de la uiversidad

## Trabajo y Obra Social


In [66]:
y_data = data_sel['nota_matematica']
x_data = data_sel[['alm_Tr' , 'alm_NoTr_B',
    'OS_sin', 'OS_voluntario', 'OS_univ',
                   'OS_trabajo' ]]
logit_st(x_data, y_data)
logit_sk(x_data, y_data)


Optimization terminated successfully.
         Current function value: 0.609785
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:        nota_matematica   No. Observations:                 2707
Model:                          Logit   Df Residuals:                     2700
Method:                           MLE   Df Model:                            6
Date:                Mon, 26 Aug 2019   Pseudo R-squ.:                 0.09358
Time:                        21:03:35   Log-Likelihood:                -1650.7
converged:                       True   LL-Null:                       -1821.1
Covariance Type:            nonrobust   LLR p-value:                 1.426e-70
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             1.0685      0.066     16.174      0.000       0.939       1.198
alm_Tr           -0.

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anac

()

### resultado Trabajo alumno y obra social
Es esperable que parte de lo que se ve a través de obra social se explique por la condición de alumno trabajador. Y eso se observa, que la dependencia con "trabajo" y con "OS_trabajo" disminuyen, porque la correlación se reparte entre ambas variables. También baja la dependencia con "no trabaja y busca" y con "OS_univ" (que deja de ser significativo) y "OS_sin". 

## Modelo de regresión logística, que intenta explicar el desempeño en matemáticas, incluyendo las variables "trabajo del alumno" y "trabajo de padre y madre" y "nivel de estudios padre y madre" y "padre y madre viven" y "nivel de inglés" y "obra social"

In [67]:
y_data = data_sel['nota_matematica']
x_data = data_sel[['alm_Tr' , 'alm_NoTr_B', 
                   'madre_vive', 'padre_vive',
                   'niv_est_madre_0', 'niv_est_madre_2','niv_est_madre_3', 'niv_est_madre_4',
                   'niv_est_padre_0', 'niv_est_padre_2','niv_est_padre_3', 'niv_est_padre_4',
                   'madre_Desconoce', 'madre_NoTr_B', 'madre_Tr',
                   'padre_Desconoce', 'padre_NoTr_B', 'padre_Tr',
                   'niv_ingles_1' ,'niv_ingles_2', 
                   'OS_sin', 'OS_voluntario', 'OS_univ',
                   'OS_trabajo' ]]
logit_st(x_data, y_data)
logit_sk(x_data, y_data)

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


Optimization terminated successfully.
         Current function value: 0.588339
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:        nota_matematica   No. Observations:                 2707
Model:                          Logit   Df Residuals:                     2682
Method:                           MLE   Df Model:                           24
Date:                Mon, 26 Aug 2019   Pseudo R-squ.:                  0.1255
Time:                        21:03:35   Log-Likelihood:                -1592.6
converged:                       True   LL-Null:                       -1821.1
Covariance Type:            nonrobust   LLR p-value:                 1.383e-81
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.6555      0.506     -1.295      0.195      -1.648       0.337
alm_Tr        

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


()

## Sacando las variables con todas las categorías no significativas (madre vive padre vive y nivel de estudios de la madre

In [68]:
y_data = data_sel['nota_matematica']
x_data = data_sel[['alm_Tr' , 'alm_NoTr_B', 
                   
                   'niv_est_madre_0', 'niv_est_madre_2','niv_est_madre_3', 'niv_est_madre_4',
                   'niv_est_padre_0', 'niv_est_padre_2','niv_est_padre_3', 'niv_est_padre_4',
                   'madre_Desconoce', 'madre_NoTr_B', 'madre_Tr',
                   
                   'niv_ingles_1' ,'niv_ingles_2', 
                   'OS_sin', 'OS_voluntario', 'OS_univ',
                   'OS_trabajo' ]]
logit_st(x_data, y_data)
logit_sk(x_data, y_data)

Optimization terminated successfully.
         Current function value: 0.589275
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:        nota_matematica   No. Observations:                 2707
Model:                          Logit   Df Residuals:                     2687
Method:                           MLE   Df Model:                           19
Date:                Mon, 26 Aug 2019   Pseudo R-squ.:                  0.1241
Time:                        21:03:35   Log-Likelihood:                -1595.2
converged:                       True   LL-Null:                       -1821.1
Covariance Type:            nonrobust   LLR p-value:                 6.657e-84
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.3303      0.340     -0.972      0.331      -0.997       0.336
alm_Tr        

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lucia/anac

()

## Métricas del modelo de regresión logística 

In [69]:
from sklearn.model_selection import train_test_split
y_data = data_sel['nota_matematica']

x_data = data_sel[['alm_Tr' , 'alm_NoTr_B', 
                   'madre_vive', 'padre_vive',
                   'niv_est_madre_0', 'niv_est_madre_2','niv_est_madre_3', 'niv_est_madre_4',
                   'niv_est_padre_0', 'niv_est_padre_2','niv_est_padre_3', 'niv_est_padre_4',
                   'madre_Desconoce', 'madre_NoTr_B', 'madre_Tr',
                   'padre_Desconoce', 'padre_NoTr_B', 'padre_Tr',
                   'niv_ingles_1' ,'niv_ingles_2', 
                   'OS_sin', 'OS_voluntario', 'OS_univ',
                   'OS_trabajo' ]]

X_train, X_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=0)


logreg = LogisticRegression(fit_intercept = True, C = 1e9)
m_fit = logreg.fit(X_train, y_train)
print(m_fit)
print('coeficientes= ',logreg.coef_)
print('intercepto= ', logreg.intercept_)

y_train_preds = logreg.predict(X_train)
y_val_preds = logreg.predict(X_val)

accuracy = metrics.accuracy_score(y_train, y_train_preds, normalize=True)
precision = metrics.precision_score(y_train, y_train_preds, pos_label=1)
recall = metrics.recall_score(y_train, y_train_preds, pos_label=1)
F1 = metrics.f1_score(y_train, y_train_preds)
conf_mx = metrics.confusion_matrix(y_train, y_train_preds)
print("Métricas del clasificador evaluado sobre el conjunto de ENTRENAMIENTO")
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", F1)
print("Matriz de Confusión: ")
print(conf_mx)

accuracy = metrics.accuracy_score(y_val, y_val_preds, normalize=True)
precision = metrics.precision_score(y_val, y_val_preds, pos_label=1)
recall = metrics.recall_score(y_val, y_val_preds, pos_label=1)
F1 = metrics.f1_score(y_val, y_val_preds)
conf_mx = metrics.confusion_matrix(y_val, y_val_preds)

print("Métricas del clasificador evaluado sobre el conjunto de EVALUACIÓN")
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", F1)
print("Matriz de Confusión: ")
print(conf_mx)

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
coeficientes=  [[-0.79768155 -0.67521606  0.07066332 -0.09625023 -1.13879164 -0.34707476
  -0.16306463 -0.00374633  0.32315181  0.34660634  0.52444816  0.79009422
   0.24772159  0.38177796  0.28039379  0.23093138  0.06255614  0.28534801
   0.3850084   0.73289517 -0.42480469  0.22343317 -0.11389106 -1.07241269]]
intercepto=  [-0.5397658]
Métricas del clasificador evaluado sobre el conjunto de ENTRENAMIENTO
Accuracy:  0.6905311778290993
Precision:  0.7047803617571059
Recall:  0.8366564417177914
F1:  0.7650771388499299
Matriz de Confusión: 
[[ 404  457]
 [ 213 1091]]
Métricas del clasificador evaluado sobre el conjunto de EVALUACIÓN
Accuracy:  0.728782287822878

In [70]:
y_data = data_sel['nota_matematica']
x_data = data_sel[['alm_Tr' , 'alm_NoTr_B', 
                   
                   'niv_est_madre_0', 'niv_est_madre_2','niv_est_madre_3', 'niv_est_madre_4',
                   'niv_est_padre_0', 'niv_est_padre_2','niv_est_padre_3', 'niv_est_padre_4',
                   'madre_Desconoce', 'madre_NoTr_B', 'madre_Tr',
                   
                   'niv_ingles_1' ,'niv_ingles_2', 
                   'OS_sin', 'OS_voluntario', 'OS_univ',
                   'OS_trabajo' ]]



X_train, X_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=0)

logreg.fit(X_train, y_train)

logreg = LogisticRegression(fit_intercept = True, C = 1e9)
m_fit = logreg.fit(X_train, y_train)
print(m_fit)
print('coeficientes= ',logreg.coef_)
print('intercepto= ', logreg.intercept_)
y_train_pred = logreg.predict(X_train)
y_val_pred = logreg.predict(X_val)

y_train_preds = logreg.predict(X_train)
y_val_preds = logreg.predict(X_val)

accuracy = metrics.accuracy_score(y_train, y_train_preds, normalize=True)
precision = metrics.precision_score(y_train, y_train_preds, pos_label=1)
recall = metrics.recall_score(y_train, y_train_preds, pos_label=1)
F1 = metrics.f1_score(y_train, y_train_preds)
conf_mx = metrics.confusion_matrix(y_train, y_train_preds)
print("Métricas del clasificador evaluado sobre el conjunto de ENTRENAMIENTO")
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", F1)
print("Matriz de Confusión: ")
print(conf_mx)

accuracy = metrics.accuracy_score(y_val, y_val_preds, normalize=True)
precision = metrics.precision_score(y_val, y_val_preds, pos_label=1)
recall = metrics.recall_score(y_val, y_val_preds, pos_label=1)
F1 = metrics.f1_score(y_val, y_val_preds)
conf_mx = metrics.confusion_matrix(y_val, y_val_preds)

print("Métricas del clasificador evaluado sobre el conjunto de EVALUACIÓN")
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", F1)
print("Matriz de Confusión: ")
print(conf_mx)

/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
coeficientes=  [[-0.79163518 -0.67966885 -1.15437003 -0.3499977  -0.16687381 -0.0142444
   0.37050906  0.35316484  0.52651362  0.80156685  0.2384468   0.39339378
   0.30635787  0.38749896  0.7414721  -0.42580045  0.22435882 -0.10939981
  -1.08688995]]
intercepto=  [-0.32090451]
Métricas del clasificador evaluado sobre el conjunto de ENTRENAMIENTO
Accuracy:  0.6863741339491917
Precision:  0.7014829142488717
Recall:  0.8343558282208589
F1:  0.762171628721541
Matriz de Confusión: 
[[ 398  463]
 [ 216 1088]]
Métricas del clasificador evaluado sobre el conjunto de EVALUACIÓN
Accuracy:  0.7250922509225092
Precision:  0.7306666666666667
Recall:  0.8509316770186336


## OTROS ALGORITMOS DE CLASIFICACION: PERCEPTRON, SGD

In [71]:
from sklearn.linear_model import LinearRegression, LogisticRegression, Perceptron, Ridge, SGDClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from sklearn import metrics
from sklearn.model_selection import GridSearchCV


In [72]:
datos_ingresantes = datos_null
print(datos_ingresantes.shape)

dic_columnas = {'alm_No trabajó y buscó trabajo en algún momento de los últimos 30 días': 'alm_NoTr_B',
'alm_No trabajó y no buscó trabajo': 'alm_NoTr_NoB',
'alm_Trabajó al menos una hora': 'alm_Tr',
'madre_No trabajó y buscó trabajo en algún momento de los últimos 30 días': 'madre_NoTr_B',
'madre_No trabajó y no buscó trabajo': 'madre_NoTr_NoB',
'madre_Trabajó al menos una hora': 'madre_Tr',
'padre_No trabajó y buscó trabajo en algún momento de los últimos 30 días': 'padre_NoTr_B',
'padre_No trabajó y no buscó trabajo': 'padre_NoTr_NoB',
'padre_Trabajó al menos una hora': 'padre_Tr',
'Carece de Obra Social' : 'OS_sin',
'Como afiliado voluntario': 'OS_voluntario',
'Otorgada por la universidad': 'OS_univ',
'Por ser familiar a cargo': 'OS_familiar',
'Por su trabajo':  'OS_trabajo'}

datos_ingresantes.rename(columns=dic_columnas, inplace=True)
datos_ingresantes.columns

(2707, 59)


Index(['nombre_carrera', 'sexo', 'obra_social', 'madre_trabajo', 'madre_vive',
       'padre_trabajo', 'padre_vive', 'trabajo', 'horas_sem_trabajo',
       'padre_ult_est_curs', 'madre_ult_est_curs', 'nota_matematica',
       'matcursprisem', 'cantmataprob_al_21-10-2017', 'dep_practica',
       'idioma_ingles', 'idioma_franc', 'idioma_portu', 'idioma_itali',
       'idioma_aleman', 'estudios_padre', 'estudios_madre', 'estudios_padre_s',
       'estudios_madre_s', 'niv_est_madre_0', 'niv_est_madre_1',
       'niv_est_madre_2', 'niv_est_madre_3', 'niv_est_madre_4',
       'niv_est_padre_0', 'niv_est_padre_1', 'niv_est_padre_2',
       'niv_est_padre_3', 'niv_est_padre_4', 'alm_NoTr_B', 'alm_NoTr_NoB',
       'alm_Tr', 'madre_Desconoce', 'madre_NoTr_B', 'madre_NoTr_NoB',
       'madre_Tr', 'padre_Desconoce', 'padre_NoTr_B', 'padre_NoTr_NoB',
       'padre_Tr', 'OS_sin', 'OS_voluntario', 'OS_univ', 'OS_familiar',
       'OS_trabajo', 'alm_35 o mas horas', 'alm_Hasta 10 horas',
       'alm_

Vamos a usar un 80% de los datos para entrenamiento y un 20% para validación.

In [73]:
train_set = datos_ingresantes.sample(frac=0.8, random_state=0)
test_set = datos_ingresantes.drop(train_set.index)

In [74]:
#data = ['estudios_madre', 'estudios_padre','alm_No trabajó y buscó trabajo en algún momento de los últimos 30 días',
#       'alm_No trabajó y no buscó trabajo', 'alm_Trabajó al menos una hora','otros_idms','idioma_ingles']


def sel_datos(train_set, test_set, data):
    X_train = train_set[data]
    X_val = test_set[data]

    y_train = train_set['nota_matematica']
    y_val = test_set['nota_matematica']

    feature_map = {feature: idx for idx, feature in enumerate(datos_ingresantes.columns)}
    return(X_train, y_train, X_val, y_val)

In [75]:
def cperceptron(X_train, y_train, X_val, y_val):
    penalty = 'l2' # TODO: Tipo de regularización: l1 (valor absoluto), l2 (cuadrados), elasticnet (l1 + l2).
    alpha = 0.1
    # TODO: Parámetro de regularización. También denominado como parámetro `lambda`.
    max_iter = 100
    # TODO: Cantidad máxima de iteraciones del algoritmo

    model = Perceptron(penalty=penalty, alpha=alpha, max_iter=max_iter)
    model.fit(X_train, y_train)

    # Evaluamos el desempeño del clasificador utilizando la exactitud (accuracy) sobre el conjunto
    # de datos de entrenamiento (X_train, y_train) y lo comparamos con el de validación (X_val, y_val)
    # La exactitud toma valor en el rango [0, 1] donde más alto es mejor
    print('Exactitud para entrenamiento: %.2f' %  accuracy_score(y_train, model.predict(X_train)))
    print('Exactitud para validación: %.2f' % accuracy_score(y_val, model.predict(X_val)))

In [76]:
data = ['estudios_madre', 'estudios_padre','alm_NoTr_B', 'alm_NoTr_NoB', 'alm_Tr',
        'otros_idms','idioma_ingles']
X_train, y_train, X_val, y_val = sel_datos(train_set, test_set, data)
cperceptron(X_train, y_train, X_val, y_val)

Exactitud para entrenamiento: 0.61
Exactitud para validación: 0.61


Usamos el conjunto de variables que se usaron para regresión logística

In [77]:
data_l = ['alm_Tr' , 'alm_NoTr_B', 
                   'madre_vive', 'padre_vive',
                   'niv_est_madre_0', 'niv_est_madre_2','niv_est_madre_3', 'niv_est_madre_4',
                   'niv_est_padre_0', 'niv_est_padre_2','niv_est_padre_3', 'niv_est_padre_4',
                   'madre_Desconoce', 'madre_NoTr_B', 'madre_Tr',
                   'padre_Desconoce', 'padre_NoTr_B', 'padre_Tr',
                   'niv_ingles_1' ,'niv_ingles_2', 
                   'OS_sin', 'OS_voluntario', 'OS_univ',
                   'OS_trabajo' ]
X_train_l, y_train_l, X_val_l, y_val_l = sel_datos(train_set, test_set, data_l)
cperceptron(X_train_l, y_train_l, X_val_l, y_val_l)

Exactitud para entrenamiento: 0.60
Exactitud para validación: 0.61


Utilizando solamente los estudios del padre y madre, no obtenemos muy buena presicion en la prediccion. Vamos a considerar 
cambiar el clasificador, y ademas agregar atributos

In [78]:
def mod_sgd(model, X_train, y_train, X_val, y_val ):
    #model = SGDClassifier(random_state=0)
    model.fit(X_train, y_train)

    preds = model.predict(X_train)
    accuracy = metrics.accuracy_score(y_train, preds, normalize=True)
    precision = metrics.precision_score(y_train, preds, pos_label=1)
    recall = metrics.recall_score(y_train, preds, pos_label=1)
    F1 = metrics.f1_score(y_train, preds)
    conf_mx = metrics.confusion_matrix(y_train, preds)
    print("Métricas del clasificador evaluado sobre el conjunto de ENTRENAMIENTO")
    print("Accuracy: ", accuracy)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1: ", F1)
    print("Matriz de Confusión: ")
    print(conf_mx)

    preds = model.predict(X_val)
    accuracy = metrics.accuracy_score(y_val, preds, normalize=True)
    precision = metrics.precision_score(y_val, preds, pos_label=1)
    recall = metrics.recall_score(y_val, preds, pos_label=1)
    F1 = metrics.f1_score(y_val, preds)
    conf_mx = metrics.confusion_matrix(y_val, preds)

    print("Métricas del clasificador evaluado sobre el conjunto de EVALUACIÓN")
    print("Accuracy: ", accuracy)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1: ", F1)
    print("Matriz de Confusión: ")
    print(conf_mx)
    model.get_params()

In [79]:
X_train, y_train, X_val, y_val = sel_datos(train_set, test_set, data)
mod_sgd(SGDClassifier(random_state=0), X_train, y_train, X_val, y_val)

Métricas del clasificador evaluado sobre el conjunto de ENTRENAMIENTO
Accuracy:  0.669898430286242
Precision:  0.6850793650793651
Recall:  0.8312788906009245
F1:  0.751131221719457
Matriz de Confusión: 
[[ 372  496]
 [ 219 1079]]
Métricas del clasificador evaluado sobre el conjunto de EVALUACIÓN
Accuracy:  0.6746765249537893
Precision:  0.7021276595744681
Recall:  0.8048780487804879
F1:  0.7500000000000001
Matriz de Confusión: 
[[101 112]
 [ 64 264]]


In [80]:
# Conjunto de datos usados en la regresión logística
X_train_l, y_train_l, X_val_l, y_val_l = sel_datos(train_set, test_set, data_l)
mod_sgd(SGDClassifier(random_state=0), X_train_l, y_train_l, X_val_l, y_val_l)

Métricas del clasificador evaluado sobre el conjunto de ENTRENAMIENTO
Accuracy:  0.6588180978762697
Precision:  0.7533998186763372
Recall:  0.6402157164869029
F1:  0.6922115785089547
Matriz de Confusión: 
[[596 272]
 [467 831]]
Métricas del clasificador evaluado sobre el conjunto de EVALUACIÓN
Accuracy:  0.6414048059149723
Precision:  0.75
Recall:  0.6128048780487805
F1:  0.674496644295302
Matriz de Confusión: 
[[146  67]
 [127 201]]


Las preguntas que nos hicimos y que nos podemos hacer, son qué tanto influyen ciertos atributos si queremos
predecir el rendimiento academico de un estudiante (medido en la nota de matematica).
Hay preguntas que no podriamos contestar con las herramientas presentadas en el curso introductorio a machine learning, 
como por ejemplo preguntas sobre si hay grupos de estudiantes con similares caracteristicas. 
Tampoco tiene mucho sentido intentar predecir funciones sobre estos datos, ya que no tenemos variables
verdaderamente continuas.
Hemos decidido limitarnos a tomar la variable binaria aprobado o desaprobado en matematica como nuestro objetivo
de prediccion, e intentar clasificar lo mejor posible a los estudiantes, probando distintos conjuntos de datos como
features para entrenar el modelo.

In [81]:
params = {
          'random_state': [0],
          'alpha': [0.001, 0.01, 0.1],
          'eta0': [0.15],
          'max_iter': [1000,2000,3000],
          'tol': [1e-3],
          'epsilon': [0.1, 0.01, 0.001],
          'l1_ratio': [0.01,0.15, 0.5, 0.99],
          'learning_rate': ['constant', 'optimal'],
          'penalty': ['none', 'l1','l2'],
          'loss': ['hinge', 'log','perceptron']
         }

scores = ['accuracy']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SGDClassifier(), params, cv=5,
                       scoring=score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    

# Tuning hyper-parameters for accuracy

Best parameters set found on development set:

{'alpha': 0.1, 'epsilon': 0.1, 'eta0': 0.15, 'l1_ratio': 0.01, 'learning_rate': 'optimal', 'loss': 'log', 'max_iter': 1000, 'penalty': 'none', 'random_state': 0, 'tol': 0.001}



/home/lucia/anaconda3/envs/diplodatos/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [82]:
dic_params = gridsearch(X_train, y_train)
best_clf = SGDClassifier(**dic_params)
mod_sgd(best_clf, X_train, y_train, X_val, y_val)

NameError: name 'gridsearch' is not defined

In [568]:
# Conjunto de datos usados en la regresión logística
dic_params = gridsearch(X_train_l, y_train_l)
best_clf = SGDClassifier(**dic_params)
mod_sgd(best_clf, X_train_l, y_train_l, X_val_l, y_val_l)

# Tuning hyper-parameters for accuracy

Best parameters set found on development set:

{'alpha': 0.001, 'epsilon': 0.1, 'eta0': 0.15, 'l1_ratio': 0.01, 'learning_rate': 'optimal', 'loss': 'log', 'max_iter': 100, 'penalty': 'l2', 'random_state': 0, 'tol': 0.001}
<class 'dict'>

Métricas del clasificador evaluado sobre el conjunto de ENTRENAMIENTO
Accuracy:  0.6943674976915974
Precision:  0.7022900763358778
Recall:  0.8505392912172574
F1:  0.7693379790940768
Matriz de Confusión: 
[[ 400  468]
 [ 194 1104]]
Métricas del clasificador evaluado sobre el conjunto de EVALUACIÓN
Accuracy:  0.6820702402957486
Precision:  0.6989795918367347
Recall:  0.8353658536585366
F1:  0.761111111111111
Matriz de Confusión: 
[[ 95 118]
 [ 54 274]]
